In [1]:
from transformers import T5Tokenizer, T5EncoderModel
import torch
import numpy as np
import pandas as pd
import umap
import base64
from io import BytesIO

/Users/akshayuppal/.pyenv/versions/3.12.8/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_excel('steroids_11_25_24 (1).xlsx', engine='openpyxl')
df.head()

,Entry,Entry Name,Protein names,Gene Names,Organism,Length,Sequence,Annotation,source,ChEBI ID,Rhea ID,SMILES
0,A0A016VA76,A0A016VA76_9BILA,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,Acey_s0014.g2217 Acey-daf-22 Y032_0014g2217,Ancylostoma ceylanicum,531,MTKPKVFVIGVGMTKFCKPGSRDWDYPDMVKEAVTTALDDCKLKYS...,5,ChEBI,17759,62960,[H][C@@]1(CC[C@@]2([H])C3=CC=C4C[C@@H](O)CC[C@...
1,A0A023EUU2,A0A023EUU2_AEDAL,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,NaN,Aedes albopictus (Asian tiger mosquito) (Stego...,544,MGVPKVYVVGVGMTKFEKPGRRENFDYPQMAKEAVTKALNDARIQY...,5,ChEBI,17759,62960,[H][C@@]1(CC[C@@]2([H])C3=CC=C4C[C@@H](O)CC[C@...
2,A0A023EUU7,A0A023EUU7_AEDAL,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,NaN,Aedes albopictus (Asian tiger mosquito) (Stego...,544,MGVPKVYVVGVGMTKFEKPGRRENFDYPQMAKEAVTKALNDARIQY...,5,ChEBI,17759,62960,[H][C@@]1(CC[C@@]2([H])C3=CC=C4C[C@@H](O)CC[C@...
3,A0A023FCJ4,A0A023FCJ4_TRIIF,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,NaN,Triatoma infestans (Assassin bug),540,RVKVYVVGVGMTKFYKPGKSDKDYPELAKEAIMKALEDARINHDDV...,5,ChEBI,17759,62960,[H][C@@]1(CC[C@@]2([H])C3=CC=C4C[C@@H](O)CC[C@...
4,A0A026WVM4,A0A026WVM4_OOCBI,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,X777_13892,Ooceraea biroi (Clonal raider ant) (Cerapachys...,541,MVYKPKVYVIGVGMTKFEKPGRRDDFDYPQMAKEAVSKALQDSRIY...,5,ChEBI,17759,62960,[H][C@@]1(CC[C@@]2([H])C3=CC=C4C[C@@H](O)CC[C@...


In [7]:
def format_sequence(seq):
    return ' '.join(seq.upper()) if isinstance(seq, str) else None

df['formatted_seq'] = df['Sequence'].apply(format_sequence)
df.head()

,Entry,Entry Name,Protein names,Gene Names,Organism,Length,Sequence,Annotation,source,ChEBI ID,Rhea ID,SMILES,formatted_seq
0,A0A016VA76,A0A016VA76_9BILA,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,Acey_s0014.g2217 Acey-daf-22 Y032_0014g2217,Ancylostoma ceylanicum,531,MTKPKVFVIGVGMTKFCKPGSRDWDYPDMVKEAVTTALDDCKLKYS...,5,ChEBI,17759,62960,[H][C@@]1(CC[C@@]2([H])C3=CC=C4C[C@@H](O)CC[C@...,M T K P K V F V I G V G M T K F C K P G S R D ...
1,A0A023EUU2,A0A023EUU2_AEDAL,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,NaN,Aedes albopictus (Asian tiger mosquito) (Stego...,544,MGVPKVYVVGVGMTKFEKPGRRENFDYPQMAKEAVTKALNDARIQY...,5,ChEBI,17759,62960,[H][C@@]1(CC[C@@]2([H])C3=CC=C4C[C@@H](O)CC[C@...,M G V P K V Y V V G V G M T K F E K P G R R E ...
2,A0A023EUU7,A0A023EUU7_AEDAL,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,NaN,Aedes albopictus (Asian tiger mosquito) (Stego...,544,MGVPKVYVVGVGMTKFEKPGRRENFDYPQMAKEAVTKALNDARIQY...,5,ChEBI,17759,62960,[H][C@@]1(CC[C@@]2([H])C3=CC=C4C[C@@H](O)CC[C@...,M G V P K V Y V V G V G M T K F E K P G R R E ...
3,A0A023FCJ4,A0A023FCJ4_TRIIF,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,NaN,Triatoma infestans (Assassin bug),540,RVKVYVVGVGMTKFYKPGKSDKDYPELAKEAIMKALEDARINHDDV...,5,ChEBI,17759,62960,[H][C@@]1(CC[C@@]2([H])C3=CC=C4C[C@@H](O)CC[C@...,R V K V Y V V G V G M T K F Y K P G K S D K D ...
4,A0A026WVM4,A0A026WVM4_OOCBI,Sterol carrier protein 2 (EC 2.3.1.155) (EC 2....,X777_13892,Ooceraea biroi (Clonal raider ant) (Cerapachys...,541,MVYKPKVYVIGVGMTKFEKPGRRDDFDYPQMAKEAVSKALQDSRIY...,5,ChEBI,17759,62960,[H][C@@]1(CC[C@@]2([H])C3=CC=C4C[C@@H](O)CC[C@...,M V Y K P K V Y V I G V G M T K F E K P G R R ...


In [8]:
# Load ProtT5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False)
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")
model = model.eval()  # Set to eval mode


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
embeddings = []
batch_size = 16

sequences = df['formatted_seq'].dropna().tolist()

for i in range(0, len(sequences), batch_size):
    batch = sequences[i:i+batch_size]
    tokens = tokenizer(batch, return_tensors='pt', padding=True, truncation=True)
    
    with torch.no_grad():
        output = model(**tokens)
    
    # Mean-pool over sequence length to get one vector per sequence
    batch_embeddings = output.last_hidden_state.mean(dim=1)
    embeddings.extend(batch_embeddings.cpu().numpy())


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


: 